<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# Cours TAL - Laboratoire 5
# Applications du modèle word2vec

**Objectifs**

Comparer des modèles word2vec pré-entraînés sur l’anglais avec des modèles appris localement sur
deux corpus, en les appliquant à des tâches de mesures de similarité et d’analogie entre mots.

Consignes
* Soumettre sur Cyberlearn un notebook Jupyter avec les expériences, les résultats obtenus et leur
analyse. Bien présenter les étapes suivies, et répondre clairement aux questions posées dans
l’énoncé. Bien préciser les données et les commandes utilisées.
* Le travail est à effectuer en binôme.
* Ne pas hésiter à consulter la documentation de Gensim sur word2vec, ainsi que celle sur les
KeyedVectors, qui forment une classe plus générale avec plusieurs exemples intéressants.
* Les différentes tâches se feront soit sur votre propre ordinateur (si vous disposez d’au moins 16
Go de RAM), soit sur un notebook fourni par le service Google Colab.

## Partie 1. Tester et évaluer un modèle entraîné sur Google News


a. Installez gensim, la bibliothèque implémentant les outils pour travailler avec Word2Vec ( pip
install --upgrade gensim ). Puis chargez le modèle word2vec pré-entraîné sur le corpus
Google News en écrivant : w2v_model = gensim.downloader.load("word2vec-google-news-
300") , ce qui télécharge le fichier la première fois, et enfin en ne gardant que les vecteurs de
mots, avec « w2v_vectors = w2v_model.wv » puis « del w2v_model » ).
Si on dispose du fichier en local, on peut le charger en écrivant w2v_vectors =
KeyedVectors.load_word2vec_format(path_to_file, binary=True) . 

### Quelle place mémoire occupe le processus du notebook une fois les vecteurs de mots chargés ?

In [1]:
import gensim.downloader as api

unable to import 'smart_open.gcs', disabling that module


In [ ]:

w2v_model = api.load("word2vec-google-news-300")

In [ ]:
w2v_vectors = w2v_model.wv
del w2v_model

Le processus du notebook a l'air d'occuper 4.5 GiB

#### b. Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ? Et quelle est la taille du vocabulaire du modèle ? Identifiez cinq mots qui sont dans le vocabulaire et un qui ne l’est pas.

In [ ]:
from gensim.test.utils import get_tmpfile, common_texts

fname = get_tmpfile("vectors.kv")
w2v_vectors.save(fname)

In [ ]:
w2v_vectors = KeyedVectors.load(fname, mmap='r')

In [ ]:
print('Pizza: ', len(w2v_vectors['Pizza']))
print('Spaghetti: ', len(w2v_vectors['Spaghetti']))
print('Hamburger: ', len(w2v_vectors['Hamburger']))
print('Cheesecake: ', len(w2v_vectors['Cheesecake']))
print('Brownie: ', len(w2v_vectors['Brownie']))

La dimension de l'espace vectoriel d'un vecteur semble être **300**.

In [ ]:
try:
  print('COVID-19: ', len(w2v_vectors['COVID-19']))
except:
  print("Not in vocabulary") 

In [ ]:
# Vocabulary size

print("La taille est de: ", len(w2v_vectors.vocab.keys()))

### c. Comment peut-on mesurer la distance entre deux mots dans cet espace ? Calculez par exemple la distance entre les mots rabbit et carrot.

In [ ]:
def dist_w2v(word1, word2):
    return w2v_vectors.distance(word1, word2)

distance = dist_w2v("rabbit", "carrot")
print("{:.1f}".format(distance))

### d. Testez le modèle de distance entre mots. Est-ce que des mots proches sémantiquement sont aussi proches dans l’espace vectoriel, et inversement ? Testez au moins cinq paires de mots. 

In [ ]:
from itertools import combinations

def eval_dist(words):
    comb = list(combinations(words, r=2))

    for p in comb: 
        distance = dist_w2v(p[0], p[1])
        print("Distance between: ", p, "{:.1f}".format(distance))

In [ ]:
some_words = ["booze", "alcohol", "brew", "wine", "cocktail", "beer", "ethanol"]

eval_dist(some_words)

Alcohol et ethanol sont techniquement la même chose, mais ont une distance très élevée.
Le sens scientifique éloigne probablement. 

Wine, beer et booze sont proches. Brew et beer aussi. 

### e. Y a-t-il des cas ambigus, et pourquoi selon vous ? Par exemple, pouvez-vous trouver des mots opposés selon le sens qui sont proches dans l’espace réduit ?

In [ ]:
other_words = ["joy", "sadness", "happiness", "mournful", "delight"]


eval_dist(other_words)

Joy est à la même distance de sadness et happiness. Rigolo :)

L'exemple ci-dessus démontre quelques cas amusants.

Joy est proche de happiness et delight, mais entre eux happiness et delight sont très éloignés

### f. Que dire des mots ayant plusieurs sens ? Pouvez-vous donner 3 exemples de ce problème ?

In [ ]:
other_words = ["run", "sprint", "escape", "jog", "start"]


eval_dist(other_words)

Run est réputé pour avoir plusieurs signifiés.

Ici on voit pourtant que run n'est proche d'aucun de ces synonymes. 

source: https://www.thesaurus.com/browse/run?s=t

### g. En vous aidant de la documentation de Gensim sur KeyedVectors, mesurez de manière quantitative la performance du modèle sur le corpus WordSimilarity-353. Expliquez ce que signifient vos résultats.

In [ ]:
from gensim.test.utils import datapath

In [ ]:
similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

similarities

### h. De même, en vous inspirant de la documentation, évaluez le modèle sur les données de test appelées questions-words.txt. Pouvez-vous expliquer ce que mesure ce test ? Les résultats du modèle sont-ils satisfaisants ? Commentez.

In [ ]:
analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))
analogy_scores

## 2. Entraîner et tester deux nouveaux modèles à partir de corpus